# Face Mask Detection Project

This project aims to detect whether a person is wearing a mask or not using a Convolutional Neural Network (CNN). We will use the MobileNetV2 architecture.

## 1. Import Dependencies

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# Check for GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## 2. Data Collection (Kaggle Dataset)

We use the [Face Mask Dataset](https://www.kaggle.com/omkargurav/face-mask-dataset). Ensure you have `kaggle.json` in the working directory.

In [ ]:
# Install Kaggle library if not already installed
# !pip install kaggle

# Configure Kaggle API
if os.path.exists('kaggle.json'):
    os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
    # Download dataset
    !kaggle datasets download -d omkargurav/face-mask-dataset
else:
    print("kaggle.json not found. Please upload it to the current directory to download the dataset.")

In [ ]:
import zipfile

# Extract the dataset
dataset_zip = 'face-mask-dataset.zip'
if os.path.exists(dataset_zip):
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall('data')
    print("Dataset extracted to ./data")
else:
    print(f"{dataset_zip} not found. Ensure download was successful.")

## 3. Data Preprocessing

In [ ]:
with_mask_files = []
without_mask_files = []

data_path = 'data/data' # The extraction often creates a nested 'data' folder, adjust as needed based on zip structure
if not os.path.exists(os.path.join(data_path, 'with_mask')):
    # Fallback if structure is just data/with_mask
    data_path = 'data'

with_mask_path = os.path.join(data_path, 'with_mask')
without_mask_path = os.path.join(data_path, 'without_mask')

if os.path.exists(with_mask_path):
    with_mask_files = os.listdir(with_mask_path)
    print('Number of with mask images:', len(with_mask_files))
else:
    print("Path not found:", with_mask_path)

if os.path.exists(without_mask_path):
    without_mask_files = os.listdir(without_mask_path)
    print('Number of without mask images:', len(without_mask_files))
else:
    print("Path not found:", without_mask_path)

In [ ]:
# Create Labels
# 1 - With Mask
# 0 - Without Mask

with_mask_labels = [1] * len(with_mask_files)
without_mask_labels = [0] * len(without_mask_files)

labels = with_mask_labels + without_mask_labels

print("Total labels:", len(labels))

In [ ]:
# Load Images
data = []

# Use a smaller subset if memory is an issue, or load all
# Resizing images to 128x128 for MobileNetV2 inputs

def load_images(file_list, path):
    for img_file in file_list:
        image = cv2.imread(os.path.join(path, img_file))
        if image is not None:
            image = cv2.resize(image, (128, 128))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            data.append(image)
        else:
            # If image fails to load, we should remove its corresponding label to keep lengths consistent
            # But simplest here is to just skip and assume mostly clean data
            pass

print("Loading 'with_mask' images...")
load_images(with_mask_files, with_mask_path)

print("Loading 'without_mask' images...")
load_images(without_mask_files, without_mask_path)

# Convert to numpy arrays
X = np.array(data)
Y = np.array(labels)

print("Data shape:", X.shape)
print("Labels shape:", Y.shape)

In [ ]:
# Normalize the data
X = X / 255.0

In [ ]:
# Split into Train and Test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

## 4. Model Building (MobileNetV2)

In [ ]:
num_of_classes = 2

model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_of_classes, activation='sigmoid'))  # Using sigmoid inside SparseCategoricalCrossentropy logic or softmax
# Actually for 2 classes (binary-like but treating as sparse categorical), we use softmax with 2 units or sigmoid with 1 unit.
# Let's stick to the previous user's likely approach: Softmax with 2 units
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

## 5. Training the Model

In [ ]:
history = model.fit(X_train, Y_train, validation_split=0.1, epochs=5, batch_size=32)

## 6. Evaluation

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

In [ ]:
# Plotting Loss and Accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()

## 7. Predictive System

In [ ]:
def predict_mask(image_path):
    input_image = cv2.imread(image_path)
    if input_image is None:
        print("Could not read image")
        return
        
    plt.imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    plt.show()

    input_image_resized = cv2.resize(input_image, (128,128))
    input_image_scaled = input_image_resized / 255.0
    input_image_reshaped = np.reshape(input_image_scaled, [1,128,128,3])

    input_prediction = model.predict(input_image_reshaped)
    input_pred_label = np.argmax(input_prediction)

    if input_pred_label == 1:
        print("Prediction: The person is wearing a Mask")
    else:
        print("Prediction: The person is NOT wearing a Mask")

# Example usage (replace with actual image path after running cells)
# predict_mask('data/with_mask/with_mask_1545.jpg')